In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('ch.csv')

C:\Users\sneha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (107,116,118,121,122,123,125,128,130,138,153,158,167,168,169) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
len(df)

11245

In [4]:
colname = df.columns[[107,116,118,121,122,123,125,128,130,138,153,158,167,168,169]]
print (colname)

Index(['CA6M', 'CA11A', 'CA11C', 'CA11H', 'CA11I', 'CA11J', 'CA11L', 'CA11Q',
       'CA11S', 'CA13R', 'ML6H', 'ML6R', 'ML9E', 'ML9F', 'ML9H'],
      dtype='object')


In [5]:
df.iloc[:,[107,116,118,121,122,123,125,128,130,138,153,158,167,168,169]] = df.iloc[:,[107,116,118,121,122,123,125,128,130,138,153,158,167,168,169]].astype(object)
#df.drop(df.columns[[107,116,118,121,122,123,125,128,130,138,153,158,167,168,169]], axis=1) 
df.head()

,HH1,HH2,LN,UF1,UF2,UF4,UF6,UF7,UF8D,UF8M,...,CDOI,CDOB,CAGE,CAGE_6,CAGE_11,melevel,religion,wscore,windex5,chweight
0,1,1,3,1,1,3,2,A12,14,2,...,1322,NaN,NaN,NaN,NaN,3,1,0.0000,0,0.000000
1,1,1,4,1,1,4,2,A12,14,2,...,1322,NaN,NaN,NaN,NaN,3,1,0.0000,0,0.000000
2,1,21,6,1,21,6,1,A12,15,2,...,1322,1273.0,49.0,6.0,5.0,3,99,0.7822,5,1.096132
3,1,21,7,1,21,7,1,A12,15,2,...,1322,1301.0,21.0,3.0,2.0,3,99,0.7822,5,1.096132
4,1,21,8,1,21,8,1,A12,15,2,...,1322,1317.0,5.0,1.0,1.0,3,99,0.7822,5,1.096132


In [6]:
g = df.columns.to_series().groupby(df.dtypes).groups
g

{dtype('int64'): Index(['HH1', 'HH2', 'LN', 'UF1', 'UF2', 'UF4', 'UF6', 'UF8D', 'UF8M', 'UF8Y',
        'UF9', 'UF11', 'HH6', 'HH7', 'HL4', 'PSU', 'stratum', 'CDOI', 'melevel',
        'religion', 'windex5'],
       dtype='object'),
 dtype('float64'): Index(['UF12H', 'UF12M', 'UF13H', 'UF13M', 'AG1D', 'AG1M', 'AG1Y', 'AG2',
        'BR1', 'BR1AD',
        ...
        'TN12_4', 'TN12A', 'ED4A', 'ED4B', 'CDOB', 'CAGE', 'CAGE_6', 'CAGE_11',
        'wscore', 'chweight'],
       dtype='object', length=188),
 dtype('O'): Index(['UF7', 'UF10', 'EC7AA', 'EC7AB', 'EC7AX', 'EC7AY', 'EC7BA', 'EC7BB',
        'EC7BX', 'EC7BY', 'EC7CA', 'EC7CB', 'EC7CX', 'EC7CY', 'EC7DA', 'EC7DB',
        'EC7DX', 'EC7DY', 'EC7EA', 'EC7EB', 'EC7EX', 'EC7EY', 'EC7FA', 'EC7FB',
        'EC7FX', 'EC7FY', 'CA6A', 'CA6B', 'CA6C', 'CA6G', 'CA6H', 'CA6L',
        'CA6M', 'CA6N', 'CA6O', 'CA6Q', 'CA6X', 'CA11A', 'CA11B', 'CA11C',
        'CA11D', 'CA11H', 'CA11I', 'CA11J', 'CA11K', 'CA11L', 'CA11O', 'CA11P',
        'CA11

In [7]:
df = df.loc[:, ~df.columns.str.startswith('UF')]
df = df.loc[:, ~df.columns.str.startswith('HH')]
df = df.loc[:, ~df.columns.str.startswith('LN')]
df.drop('AN1', axis=1, inplace=True)

In [8]:
target = df['IM6']
df.drop('IM6', axis=1, inplace=True)

In [9]:
df = df.loc[:, ~df.columns.str.startswith('IM')]

In [10]:
dummies = pd.get_dummies(df)
dummies.head()

,AG1D,AG1M,AG1Y,AG2,BR1,BR1AD,BR1AM,BR1AY,BR2,BR2A,...,ML9P_?,ML9P_P,ML9Q_?,ML9Q_Q,ML9R_?,ML9R_R,ML9X_?,ML9X_X,ML9Z_?,ML9Z_Z
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,4.0,1.0,2006.0,4.0,3.0,NaN,NaN,NaN,2.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,6.0,5.0,2008.0,1.0,3.0,NaN,NaN,NaN,2.0,NaN,...,0,0,0,1,0,0,0,0,0,0
4,4.0,9.0,2009.0,0.0,3.0,NaN,NaN,NaN,2.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [11]:
dummies.columns

Index(['AG1D', 'AG1M', 'AG1Y', 'AG2', 'BR1', 'BR1AD', 'BR1AM', 'BR1AY', 'BR2',
       'BR2A',
       ...
       'ML9P_?', 'ML9P_P', 'ML9Q_?', 'ML9Q_Q', 'ML9R_?', 'ML9R_R', 'ML9X_?',
       'ML9X_X', 'ML9Z_?', 'ML9Z_Z'],
      dtype='object', length=264)

In [12]:
#cols = [col for col in dummies if col.startswith('IM1_')]
#cols
dummies['recall'] = target

In [13]:
dummies['recall'].unique()

array([nan,  1.,  8.,  2.,  9.])

In [14]:
dummies['is_recall'] = dummies.recall.apply(lambda x: 1 if x == 1 else 0)
dummies.drop('recall', axis=1, inplace=True)
y=['is_recall']
dummy_vars =dummies.columns.values.tolist()
X=[i for i in dummy_vars if i not in y]

In [15]:
dummies = dummies.fillna(0)

In [16]:
logreg = LogisticRegression()
rfe = RFE(logreg,n_features_to_select= 25)
rfe = rfe.fit(dummies[X],dummies[y].values.ravel())
print(rfe.support_,rfe.ranking_)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False  True False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [17]:
cols = []
for i in range(len(rfe.ranking_)):
    if rfe.ranking_[i] == 1:
        print(i,rfe.support_[i],rfe.ranking_[i],dummies.columns[i])   
        cols.append(dummies.columns[i])

98 True 1 WHZNOAGE
122 True 1 CAGE_6
130 True 1 EC7AA_A
132 True 1 EC7AB_B
180 True 1 CA6B_B
182 True 1 CA6C_C
190 True 1 CA6M_M
201 True 1 CA11C_C
202 True 1 CA11D_D
204 True 1 CA11I_I
207 True 1 CA11L_L
210 True 1 CA11Q_Q
211 True 1 CA11R_R
212 True 1 CA11S_S
226 True 1 ML6E_E
227 True 1 ML6F_F
228 True 1 ML6H_H
239 True 1 ML9B_B
245 True 1 ML9E_E
246 True 1 ML9F_?
249 True 1 ML9H_H
250 True 1 ML9I_?
253 True 1 ML9J_J
258 True 1 ML9R_?
259 True 1 ML9R_R


In [18]:
X=dummies[cols]
y=dummies['is_recall']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.65


In [21]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=10, random_state=7)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

10-fold cross validation average accuracy: 0.636


In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[ 932  603]
 [ 570 1269]]


In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.62      0.61      0.61      1535
          1       0.68      0.69      0.68      1839

avg / total       0.65      0.65      0.65      3374

